In [1]:
import os
import librosa
import soundfile as sf

In [2]:
# Convert MP3 files to WAV format
# Define paths
mp3_dir = r'C:/Users/yingx/OneDrive/Documents/GitHub/Capstone/data/MEMD_audio'
wav_dir = r'C:/Users/yingx/OneDrive/Documents/GitHub/Capstone/data/WaveFiles'

# Create output directory if it doesn't exist
os.makedirs(wav_dir, exist_ok=True)

# Process files from 2.mp3 to 2058.mp3
for i in range(2, 2059):
    mp3_path = os.path.join(mp3_dir, f'{i}.mp3')
    wav_path = os.path.join(wav_dir, f'{i}.wav')
    
    # Only process if the MP3 file exists
    if os.path.exists(mp3_path):
        try:
            # Load MP3 file with librosa (automatically converts to 22050 Hz mono)
            y, sr = librosa.load(mp3_path, sr=None)  # sr=None keeps original sample rate
            
            # Save as WAV file using soundfile
            sf.write(wav_path, y, sr)
            
            print(f'Successfully converted {i}.mp3 to {i}.wav')
        except Exception as e:
            print(f'Error processing {i}.mp3: {str(e)}')
    else:
        print(f'{i}.mp3 not found, skipping')

print('Conversion complete!')

Successfully converted 2.mp3 to 2.wav
Successfully converted 3.mp3 to 3.wav
Successfully converted 4.mp3 to 4.wav
Successfully converted 5.mp3 to 5.wav
6.mp3 not found, skipping
Successfully converted 7.mp3 to 7.wav
Successfully converted 8.mp3 to 8.wav
9.mp3 not found, skipping
Successfully converted 10.mp3 to 10.wav
11.mp3 not found, skipping
Successfully converted 12.mp3 to 12.wav
Successfully converted 13.mp3 to 13.wav
14.mp3 not found, skipping
15.mp3 not found, skipping
16.mp3 not found, skipping
Successfully converted 17.mp3 to 17.wav
Successfully converted 18.mp3 to 18.wav
Successfully converted 19.mp3 to 19.wav
Successfully converted 20.mp3 to 20.wav
Successfully converted 21.mp3 to 21.wav
Successfully converted 22.mp3 to 22.wav
23.mp3 not found, skipping
Successfully converted 24.mp3 to 24.wav
Successfully converted 25.mp3 to 25.wav
26.mp3 not found, skipping
27.mp3 not found, skipping
28.mp3 not found, skipping
29.mp3 not found, skipping
30.mp3 not found, skipping
Successful

In [17]:
import os
import whisper
import soundfile as sf
import numpy as np
from datetime import datetime

def setup_paths():
    base_dir = r'C:\Users\yingx\OneDrive\Documents\GitHub\Capstone\data'
    wav_dir = os.path.join(base_dir, 'WaveFiles')
    output_file = os.path.join(base_dir, 'transcriptions.csv')
    
    if not os.path.exists(wav_dir):
        raise FileNotFoundError(f"Directory not found: {wav_dir}")
    return wav_dir, output_file

def load_audio_file(wav_path):
    """Load audio file using soundfile to ensure compatibility"""
    try:
        data, samplerate = sf.read(wav_path)
        # Convert to mono if stereo
        if len(data.shape) > 1:
            data = np.mean(data, axis=1)
        return data, samplerate
    except Exception as e:
        print(f"Error loading {wav_path}: {str(e)}")
        return None, None

def run_transcription():
    try:
        wav_dir, output_file = setup_paths()
        
        print("\nFound these WAV files:")
        valid_files = []
        for i in range(2, 8):
            wav_path = os.path.join(wav_dir, f"{i}.wav")
            if os.path.exists(wav_path):
                print(f"✓ {i}.wav")
                valid_files.append((i, wav_path))
            else:
                print(f"✗ {i}.wav (not found)")
        
        if not valid_files:
            raise FileNotFoundError("No valid WAV files found")
        
        print("\nLoading Whisper model...")
        model = whisper.load_model("medium")
        
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write("filename,transcription,processing_time\n")
            
            for i, wav_path in valid_files:
                print(f"\nProcessing {i}.wav...")
                start_time = datetime.now()
                
                try:
                    # Load audio using soundfile first
                    audio, sr = load_audio_file(wav_path)
                    if audio is None:
                        f.write(f"{i}.wav,\"audio_load_error\",0.00\n")
                        continue
                    
                    # Convert to float32 for Whisper
                    audio = audio.astype(np.float32)
                    
                    # Transcribe
                    result = model.transcribe(audio, fp16=False, language='en')
                    text = result["text"].strip() or "no vocal"
                    proc_time = (datetime.now() - start_time).total_seconds()
                    
                    f.write(f"{i}.wav,\"{text}\",{proc_time:.2f}\n")
                    print(f"Result: {text}")
                    print(f"Processed in {proc_time:.2f}s")
                    
                except Exception as e:
                    print(f"Error: {str(e)}")
                    f.write(f"{i}.wav,\"processing_error\",0.00\n")
        
        print("\nProcess completed successfully!")
        
    except Exception as e:
        print(f"\nFatal error: {str(e)}")

if __name__ == "__main__":
    print("Starting enhanced transcription process...")
    run_transcription()

Starting enhanced transcription process...

Found these WAV files:
✓ 2.wav
✓ 3.wav
✓ 4.wav
✓ 5.wav
✗ 6.wav (not found)
✓ 7.wav

Loading Whisper model...


100%|█████████████████████████████████████| 1.42G/1.42G [02:37<00:00, 9.70MiB/s]



Processing 2.wav...
Result: I'm gonna do it. I'm gonna do it. I'm gonna do it. I'm gonna do it. I'm gonna do it.
Processed in 50.12s

Processing 3.wav...
Result: no vocal
Processed in 65.81s

Processing 4.wav...
Result: no vocal
Processed in 20.53s

Processing 5.wav...
Result: Rream of stars He has my Thank you.
Processed in 138.09s

Processing 7.wav...
Result: You
Processed in 46.03s

Process completed successfully!
